In [5]:
MXE_ENDPOINT="https://mxe-vcluster1.kroto011.rnd.gic.ericsson.se"
MXE_DEPLOYER_ENDPOINT="https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se"
OAUTH_ENDPOINT="https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se"
ARGOCD_ENDPOINT="https://argocd-vcluster1.kroto011.rnd.gic.ericsson.se"
GITEA_ENDPOINT="https://gitea.mxe-vcluster1.kroto011.rnd.gic.ericsson.se"
INTERNAL_GITOPS_REPO=f"{GITEA_ENDPOINT}/mxe/mxe-gitops.git"
ARGOCD_ADMIN_USERNAME="argocd-admin"
ARGOCD_ADMIN_PASSWORD="admin-pass"
MXE_VERSION="2.4.0+263"

In [2]:
import os, json, subprocess, pathlib

In [3]:
def run_command(command):
    proc=subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = proc.communicate()
    if proc.returncode == 0:
        return out.decode("utf-8")
    else: 
        print(f"Subprocess failed")
        print(f'stdout: {out.decode("utf-8")}')
        print(f'stderr: {err.decode("utf-8")}')
        raise Exception(f"Subprocess failed with return code {proc.returncode}")

## Download CLI

In [8]:
ARM_REPO="https://arm.seli.gic.ericsson.se/artifactory"
if "+" in MXE_VERSION:
    MXE_CANDIDATE_VERSION=MXE_VERSION.replace("+", "-")
    CLI_URL=f"{ARM_REPO}/proj-mxe-release-generic/{MXE_CANDIDATE_VERSION}/mxe-cli-linux-{MXE_CANDIDATE_VERSION}.tgz"
else:
    CLI_URL=f" {ARM_REPO}/proj-mxe-dev-generic-local/mxe-cli-linux-{MXE_VERSION}.tgz"

print(f"Downloading mxe-cli from {CLI_URL}")

API_TOKEN=os.environ.get("DOCKER_API_TOKEN")

if API_TOKEN is None:
    raise Exception("ENV DOCKER_API_TOKEN not set")

CLI_DOWNLOAD_CMD=f"curl --header 'X-JFrog-Art-Api:${API_TOKEN}' --fail {CLI_URL} --output mxe-cli-linux.tgz"
#        tar xvzf .bob/mxe-cli-linux-${env.STABLE_MXE_VERSION}.tgz  -C ${env.PWD}/cli/;
#        rm -f .bob/mxe-cli-linux-${env.STABLE_MXE_VERSION}.tgz;

In [12]:
%%bash -s "$CLI_URL" "$API_TOKEN" "$MXE_ENDPOINT"

CLI_URL=$1
API_TOKEN=$2
MXE_ENDPOINT=$3

[ -d cli ] && rm -rf cli
mkdir -p cli 
curl --header "X-JFrog-Art-Api:${API_TOKEN}" --fail $CLI_URL --output mxe-cli-linux.tgz
tar xvzf mxe-cli-linux.tgz -C cli/
rm -f mxe-cli-linux.tgz

chmod +x cli/mxe-*

sed -i "s#<mxeEndpoint>#$MXE_ENDPOINT#g" cli/data/clusters.json 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.6M  100 22.6M    0     0  39.3M      0 --:--:-- --:--:-- --:--:-- 39.2M


mxe-deploy
mxe-model
mxe-service
mxe-training
data/clusters.json


## mxe-deploy login

## Non interactive

In [16]:
%%bash -s "$MXE_DEPLOYER_ENDPOINT" "$OAUTH_ENDPOINT" "$ARGOCD_ADMIN_USERNAME" "$ARGOCD_ADMIN_PASSWORD"
set -x 

cli/mxe-deploy login $1 --ssoMode --ssoHost $2 --username $3 --password $4

+ cli/mxe-deploy login https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se --ssoMode --ssoHost https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se --username argocd-admin --password admin-pass


Authentication successful


## /v1/package

Make sure https://gitlab.internal.ericsson.com/gaia-mass/mee/deployer-example-apps.git repo is added to ARGOCD_ENDPOINT before proceeding

Verify that this repo is shown in the Settings > Repository page 

<img src="Gitlab_repository.png">

#### Deploy Package from git

In [30]:
payload={
    "appName": "sample-nginx-app",
    "packageSource": {
        "repoURL": "https://gitlab.internal.ericsson.com/gaia-mass/mee/deployer-example-apps.git",
        "path": "nginx-sample-yaml",
        "targetRevision": "main"
    },
    "packageDestination": {
        "server" : "https://kubernetes.default.svc",
        "namespace": "mxe"
    },
    "labels": [ "issuedBy=deployer-notebook", "partOf=mxe"],
    "syncPolicy":{
        "automated":{
            "prune": False,
            "selfHeal": False,
            "allowEmpty" : False
        }
    }
}

labelsStr = "".join([ f" --label {label} " for label in payload["labels"] ])

syncPolicy = ""
if "syncPolicy" in payload:
    if "automated" in payload["syncPolicy"]:
        syncPolicy = f" --sync-policy automated "
        if "prune" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["prune"]:
            syncPolicy += f" --auto-prune "
            if "selfHeal" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["selfHeal"]:
                syncPolicy += f" --self-heal  "
                if "allowEmpty" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["allowEmpty"]:
                    syncPolicy += f" --allow-empty "

COMMAND=f"""cli/mxe-deploy package create {payload['appName']} --repo {payload['packageSource']['repoURL']} --path {payload['packageSource']['path']} 
--revision {payload['packageSource']['targetRevision']} --dest-namespace  {payload['packageDestination']['namespace']} 
--dest-server {payload['packageDestination']['server']} {labelsStr} --sync-policy automated --auto-prune --self-heal""".replace("\n", " ")

print(f"Running command: {COMMAND}")


output=run_command(COMMAND)

print(f"OUTPUT of Deploying app from git: {output}")


Running command: cli/mxe-deploy package create sample-nginx-app --repo https://gitlab.internal.ericsson.com/gaia-mass/mee/deployer-example-apps.git --path nginx-sample-yaml  --revision main --dest-namespace  mxe  --dest-server https://kubernetes.default.svc  --label issuedBy=deployer-notebook  --label partOf=mxe  --sync-policy automated --auto-prune --self-heal
OUTPUT of Deploying app from git: Application sample-nginx-app created successfully


### Verification

Check ArgoCD applications page, open sample-nginx-app application; it should show synced/healthy

<img src="Successful_deployment_from_git.png" width="800">

#### Verification from k8s

In [31]:
%%bash 

kubectl get application sample-nginx-app -n mxe
kubectl get deployment -l argocd.argoproj.io/instance=sample-nginx-app -n mxe

NAME               SYNC STATUS   HEALTH STATUS
sample-nginx-app   Synced        Healthy
NAME          READY   UP-TO-DATE   AVAILABLE   AGE
nginx-app-1   3/3     3            3           3h51m


## Deploy Tar file from local fs

Make tar files using sample helm chart helm-examples

In [32]:
%%bash 

[[ -d helm-examples ]] && rm -rf helm-examples
git clone https://github.com/helm/examples.git helm-examples
sed -i 's#repository: nginx#repository: armdockerhub.rnd.ericsson.se/nginx#g' helm-examples/charts/hello-world/values.yaml
tar -czvf helm-examples.tar.gz -C helm-examples/charts/hello-world .
# Change Chartname for tgz
sed -i 's#name: hello-world#name: hello-world-2#g' helm-examples/charts/hello-world/Chart.yaml
tar -czvf helm-examples.tgz -C helm-examples/charts/hello-world .

# Change Chartname for zip
sed -i 's#name: hello-world-2#name: hello-world-3#g' helm-examples/charts/hello-world/Chart.yaml
cd helm-examples/charts/hello-world/ && zip -r $OLDPWD/helm-examples.zip . && cd -

rm -rf archives && mkdir -p archives
mv helm-examples.tar.gz helm-examples.tgz helm-examples.zip archives/

ls archives/

Cloning into 'helm-examples'...


./
./values.yaml
./README.md
./Chart.yaml
./templates/
./templates/serviceaccount.yaml
./templates/deployment.yaml
./templates/_helpers.tpl
./templates/NOTES.txt
./templates/service.yaml
./.helmignore
./
./values.yaml
./README.md
./Chart.yaml
./templates/
./templates/serviceaccount.yaml
./templates/deployment.yaml
./templates/_helpers.tpl
./templates/NOTES.txt
./templates/service.yaml
./.helmignore
  adding: values.yaml (deflated 42%)
  adding: README.md (stored 0%)
  adding: Chart.yaml (deflated 55%)
  adding: templates/ (stored 0%)
  adding: templates/serviceaccount.yaml (deflated 48%)
  adding: templates/deployment.yaml (deflated 64%)
  adding: templates/_helpers.tpl (deflated 69%)
  adding: templates/NOTES.txt (deflated 61%)
  adding: templates/service.yaml (deflated 50%)
  adding: .helmignore (deflated 33%)
/mnt/disk0/enxxram/home/enxxram/workspace/mee/2.4/mxe-core-latest/mxe-deployer/mxe-deployer-service/notebooks
helm-examples.tar.gz
helm-examples.tgz
helm-examples.zip


### Deploy tar file

In [43]:
payload={
    "appName": "sample-hello-world-app1",
    "packageSource": {
        "repoURL": INTERNAL_GITOPS_REPO,
        "path": "sample_apps/app1/",
        "targetRevision": "master"
    },
    "packageDestination": {
        "server" : "https://kubernetes.default.svc",
        "namespace": "mxe"
    },
    "labels": [ "issuedBy=deployer-notebook"],
    "syncPolicy":{
        "automated":{
            "prune": False,
            "selfHeal": False,
            "allowEmpty" : False
        }
    },
    "syncOptions":["CreateNamespace=false"]
}

ARCHIVE_PATH=pathlib.Path("archives/helm-examples.tar.gz").absolute()

syncPolicy = ""
if "syncPolicy" in payload:
    if "automated" in payload["syncPolicy"]:
        syncPolicy = f" --sync-policy automated "
        if "prune" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["prune"]:
            syncPolicy += f" --auto-prune "
            if "selfHeal" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["selfHeal"]:
                syncPolicy += f" --self-heal  "
                if "allowEmpty" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["allowEmpty"]:
                    syncPolicy += f" --allow-empty "

syncOptions = ""
if "syncOptions" in payload:
    syncOptions = " ".join([f"--sync-option {option}" for option in payload["syncOptions"]])

COMMAND=f"""cli/mxe-deploy package create {payload['appName']} --repo {payload['packageSource']['repoURL']} 
--path {payload['packageSource']['path']} --revision {payload['packageSource']['targetRevision']} 
--dest-namespace  {payload['packageDestination']['namespace']} --dest-server {payload['packageDestination']['server']} 
--manifest-archive {ARCHIVE_PATH} {labelsStr} {syncPolicy} {syncOptions}""".replace("\n", " ")

print(f"\n\nDeploy tar.gz archive CLI command: {COMMAND}")

output=run_command(COMMAND)

print(f"\n\nDeploy tar.gz archive Output: {output}", flush=True)



Deploy tar.gz archive CLI command: cli/mxe-deploy package create sample-hello-world-app1 --repo https://gitea.mxe-vcluster1.kroto011.rnd.gic.ericsson.se/mxe/mxe-gitops.git  --path sample_apps/app1/ --revision master  --dest-namespace  mxe --dest-server https://kubernetes.default.svc  --manifest-archive /mnt/disk0/enxxram/home/enxxram/workspace/mee/2.4/mxe-core-latest/mxe-deployer/mxe-deployer-service/notebooks/archives/helm-examples.tar.gz  --label issuedBy=deployer-notebook  --label partOf=mxe   --sync-policy automated  --sync-option CreateNamespace=false


Deploy tar.gz archive Output: Application sample-hello-world-app1 created successfully


### Patch app

In [44]:

payload={
    "name": "sample-hello-world-app1",
}

ARCHIVE_PATH=pathlib.Path("archives/helm-examples.tgz").absolute()

COMMAND=f"cli/mxe-deploy package patch {payload['name']} --manifest-archive {ARCHIVE_PATH}"

print(f"\n\Patch app using archive Curl command: {COMMAND}")

output=run_command(COMMAND)

print(f"\n\nPatch app using archive Output: {output}", flush=True)


\Patch app using archive Curl command: cli/mxe-deploy package patch sample-hello-world-app1 --manifest-archive /mnt/disk0/enxxram/home/enxxram/workspace/mee/2.4/mxe-core-latest/mxe-deployer/mxe-deployer-service/notebooks/archives/helm-examples.tgz


Patch app using archive Output: Application sample-hello-world-app1 patched successfully


_*Verification*_ : Check that `chartName` got updated in `Chart.yaml` gitea repo `sample_apps/app1` dir

### Delete app

In [41]:
def del_app(app_to_delete):
    COMMAND=f'echo y | cli/mxe-deploy package delete {app_to_delete} -p background'

    print(f"\n\nDelete app command: {COMMAND}")

    output=run_command(COMMAND)
    print(f"\n\nDelete app Output: {output}")

In [45]:
del_app(app_to_delete="sample-hello-world-app1")



Delete app command: echo y | cli/mxe-deploy package delete sample-hello-world-app1 -p background


Delete app Output: Do you really want to delete sample-hello-world-app1 from server https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se [Enter y/n]: 
 Successfully deleted sample-hello-world-app1


## Deploy zip archive

In [46]:
payload={
    "appName": "sample-hello-world-app2",
    "packageSource": {
        "repoURL": INTERNAL_GITOPS_REPO,
        "path": "sample_apps/app2/",
        "targetRevision": "master"
    },
    "packageDestination": {
        "server" : "https://kubernetes.default.svc",
        "namespace": "mxe"
    },
    "labels": [ "issuedBy=deployer-notebook"],
    "syncPolicy":{
        "automated":{
            "prune": False,
            "selfHeal": False,
            "allowEmpty" : False
        }
    },
    "syncOptions":["CreateNamespace=false"]
}

ARCHIVE_PATH=pathlib.Path("archives/helm-examples.zip").absolute()

syncPolicy = ""
if "syncPolicy" in payload:
    if "automated" in payload["syncPolicy"]:
        syncPolicy = f" --sync-policy automated "
        if "prune" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["prune"]:
            syncPolicy += f" --auto-prune "
            if "selfHeal" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["selfHeal"]:
                syncPolicy += f" --self-heal  "
                if "allowEmpty" in payload["syncPolicy"]["automated"] and payload["syncPolicy"]["automated"]["allowEmpty"]:
                    syncPolicy += f" --allow-empty "

syncOptions = ""
if "syncOptions" in payload:
    syncOptions = " ".join([f"--sync-option {option}" for option in payload["syncOptions"]])

COMMAND=f"""cli/mxe-deploy package create {payload['appName']} --repo {payload['packageSource']['repoURL']} 
--path {payload['packageSource']['path']} --revision {payload['packageSource']['targetRevision']} 
--dest-namespace  {payload['packageDestination']['namespace']} --dest-server {payload['packageDestination']['server']} 
--manifest-archive {ARCHIVE_PATH} {labelsStr} {syncPolicy} {syncOptions}""".replace("\n", " ")

print(f"\n\nDeploy zip archive CLI command: {COMMAND}")

output=run_command(COMMAND)

print(f"\n\nDeploy zip archive Output: {output}", flush=True)



Deploy zip archive CLI command: cli/mxe-deploy package create sample-hello-world-app2 --repo https://gitea.mxe-vcluster1.kroto011.rnd.gic.ericsson.se/mxe/mxe-gitops.git  --path sample_apps/app2/ --revision master  --dest-namespace  mxe --dest-server https://kubernetes.default.svc  --manifest-archive /mnt/disk0/enxxram/home/enxxram/workspace/mee/2.4/mxe-core-latest/mxe-deployer/mxe-deployer-service/notebooks/archives/helm-examples.zip  --label issuedBy=deployer-notebook  --label partOf=mxe   --sync-policy automated  --sync-option CreateNamespace=false


Deploy zip archive Output: Application sample-hello-world-app2 created successfully


### Get Applications

#### Get app Names alone

In [50]:

COMMAND="cli/mxe-deploy package list -o name" 

OUTPUT=run_command(COMMAND)

print(f"Deployed Apps:\n{OUTPUT}", flush=True)


Deployed Apps:
sample-hello-world-app2
sample-nginx-app



#### Get all apps as json

In [51]:
COMMAND="cli/mxe-deploy package list -o json" 

OUTPUT=run_command(COMMAND)

print(f"Deployed Apps:\n{OUTPUT}", flush=True)


Deployed Apps:
[
  {
    "metadata": {
      "name": "sample-hello-world-app2",
      "namespace": "mxe",
      "uid": "3cd5b8c2-4c43-447b-af93-dcfdaed8d38e",
      "resourceVersion": "7073556",
      "generation": 12,
      "creationTimestamp": "2022-09-12T11:12:48Z",
      "labels": {
        "issuedBy": "deployer-notebook",
        "managedBy": "eric-mxe-deployer-service",
        "partOf": "mxe"
      },
      "annotations": {
        "argocd.argoproj.io/manifest-generate-paths": "/sample_apps/app2/"
      },
      "managedFields": [
        {
          "manager": "Go-http-client",
          "operation": "Update",
          "apiVersion": "argoproj.io/v1alpha1",
          "time": "2022-09-12T11:12:49Z",
          "fieldsType": "FieldsV1",
          "fieldsV1": {
            "f:metadata": {
              "f:annotations": {
                ".": {},
                "f:argocd.argoproj.io/manifest-generate-paths": {}
              },
              "f:labels": {
                ".": {},
 

### App sync command

In [58]:
def sync(app_name, force=False, prune=False):
    force = "--force" if force else ""
    prune = "--prune" if prune else ""
    COMMAND=f"cli/mxe-deploy package sync {app_name} {force} {prune}" 
    print("\n Running sync command: ", COMMAND)
    OUTPUT=run_command(COMMAND)
    print(f"App {app_name} sync status:\n{OUTPUT}", flush=True)


In [59]:
sync(app_name="sample-hello-world-app2", force=True, prune=True)


 Running sync command:  cli/mxe-deploy package sync sample-hello-world-app2 --force --prune
App sample-hello-world-app2 sync status:
Sync for App: sample-hello-world-app2  has been initiated successfully



### Wait until app has synced/healthy status

In [60]:
def wait(app_name, check_interval=10, max_attemps=2):
    COMMAND=f"cli/mxe-deploy package wait {app_name} --check-interval {check_interval} --max-attempts {max_attemps}" 
    print("\n Running wait command: ", COMMAND)
    OUTPUT=run_command(COMMAND)
    print(f"App {app_name} wait status:\n{OUTPUT}", flush=True)

In [61]:
wait(app_name="sample-hello-world-app2", check_interval=5, max_attemps=10)


 Running wait command:  cli/mxe-deploy package wait sample-hello-world-app2 --check-interval 5 --max-attempts 10
App sample-hello-world-app2 wait status:
Monitoring status of sample-hello-world-app2


Waiting for 5 seconds
Time:               2022-09-12T13:21:19+02:00
Iteration           1
Application:        sample-hello-world-app2
SyncStatus:         Synced
HealthStatus:       Healthy
Conditions:         <none>
OperationState:
	Operation:          Sync
	Sync Revision:      945c7f6ab39a15f7fea03224960e9349af46cd51
	Phase:              Succeeded
	Start:              2022-09-12 13:19:00 +0200 CEST
	Finished:           2022-09-12 13:19:00 +0200 CEST
	Duration:           0s
	Message:            successfully synced (all tasks run)


 Application sample-hello-world-app2 is synced and healthy


Waiting for 5 seconds

